# Векторные представления слов.

In [1]:
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_distances
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

Загрузим данные из тех же файлов.

In [2]:
sents = open('data/train_pos.out').read().split('\n\n') + \
        open('data/test_pos.out').read().split('\n\n')


Для word2vec'а лучше использовать лемматизированные слова, а для fasttext'а подойдут и сырые слова.

In [3]:
# сохраним целые предложения на будущее
sents_ = []

# не будем учитывать слова короче 4 (так заодно выкинтся все знаки препинания)
lemmas = []
for sent in sents:
    sent_lemmas = []
    for line in sent.split('\n'):
        lemma = line.split('\t')[1]
        if len(lemma) > 3:
            sent_lemmas.append(lemma.lower())
    lemmas.append(sent_lemmas)

raw = []
for sent in sents:
    sent_lemmas = []
    s = []
    for line in sent.split('\n'):
        lemma = line.split('\t')[0]
        s.append(lemma)
        if len(lemma) > 3:
            sent_lemmas.append(lemma.lower())
    raw.append(sent_lemmas)
    sents_.append(s)

In [ ]:
len(lemmas)

In [ ]:
lemmas[:10]

In [ ]:
# посмотрим параметры
?gensim.models.Word2Vec

In [4]:
w2v = gensim.models.Word2Vec(lemmas, size=200, max_vocab_size=100000)

In [5]:
# проверим что за модель получилась, посмотрев на близкие слова
w2v.wv.most_similar('мужчина')

[('женщина', 0.9966326951980591),
 ('представлять', 0.9890692234039307),
 ('казаться', 0.9889426827430725),
 ('позволить', 0.9874495267868042),
 ('вообразить', 0.9870392084121704),
 ('комфортно', 0.9853262901306152),
 ('ребёнок', 0.9849750995635986),
 ('душа', 0.984269380569458),
 ('обманывать', 0.9840550422668457),
 ('красивый', 0.983880877494812)]

In [6]:
# посмотрим параметры
?gensim.models.FastText

In [7]:
ft = gensim.models.FastText(raw, min_n=3, max_n=6, sg=1, size=100, max_vocab_size=100000)

In [8]:
# проверим что за модель получилась, посмотрев на близкие слова
ft.wv.most_similar('варенье')

[('кухня', 0.9960958361625671),
 ('вдаль', 0.9941439628601074),
 ('чуть_не', 0.993897557258606),
 ('папу', 0.9938607215881348),
 ('кухню', 0.9933991432189941),
 ('спирт', 0.9931646585464478),
 ('звонко', 0.9929766654968262),
 ('капусту', 0.9929111003875732),
 ('неподалеку', 0.9927726984024048),
 ('папа', 0.9926959276199341)]

### Как ни странно, но из одельных векторов слов можно сделать хорошие представления целых предложений.

Для этого просто усредним векторы отдельных векторов.

In [9]:
X = np.zeros((len(lemmas), 200))

for i, sent in enumerate(lemmas):
    try:
        vectors = w2v[sent]
    except (KeyError, ValueError):
        continue
    
    average_vector = np.mean(vectors, axis=0)
    X[i] = average_vector

Можно также искать похожие предложения с помощью косинусной близости (или каких-то других метрик)

In [10]:
sents_[26]

['Со', 'времени', 'ее', 'отъезда', 'прошло', 'три', 'года', '.']

In [11]:
cosine_distances(X[26], X).argsort()

array([[   26, 67394, 35566, ..., 60574, 32887, 41573]])

In [12]:
sents_[54970]

['С', 'тех', 'пор', 'прошло', 'немало', 'времени', '.']

Для наглядности кластеризуем предложения и посмотрим какие получаются кластеры.

In [13]:
km = MiniBatchKMeans(1000, verbose=1)
km.fit(X)

Init 1/3 with method: k-means++
Inertia for init 1/3: 1.092780
Init 2/3 with method: k-means++
Inertia for init 2/3: 0.992682
Init 3/3 with method: k-means++
Inertia for init 3/3: 0.648972
Minibatch iteration 1/83200: mean batch inertia: 0.035449, ewa inertia: 0.035449 
Minibatch iteration 2/83200: mean batch inertia: 0.043664, ewa inertia: 0.035469 
Minibatch iteration 3/83200: mean batch inertia: 0.042421, ewa inertia: 0.035486 
Minibatch iteration 4/83200: mean batch inertia: 0.036229, ewa inertia: 0.035488 
Minibatch iteration 5/83200: mean batch inertia: 0.032004, ewa inertia: 0.035479 
Minibatch iteration 6/83200: mean batch inertia: 0.036316, ewa inertia: 0.035481 
Minibatch iteration 7/83200: mean batch inertia: 0.034248, ewa inertia: 0.035478 
Minibatch iteration 8/83200: mean batch inertia: 0.030571, ewa inertia: 0.035467 
Minibatch iteration 9/83200: mean batch inertia: 0.038757, ewa inertia: 0.035474 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 10/

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=1000, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=1)

In [14]:
f = open('clustes_.txt', 'w')

labels = km.labels_ # в km.labels_ тут хранятся кластеры для каждого примера

clusters = defaultdict(list)
for i in range(len(sents_)):
    clusters[labels[i]].append(sents_[i])

for cluster in clusters:
    f.write('\n\n\nCLUSTER - {}\n'.format(cluster))
    for sent in clusters[cluster]:
        f.write(' '.join(sent) + '\n')
f.close()
    

Простое усреднение работает очень хорошо. Если найти корпус побольше и подобрать параметры, векторы предложения будут очень хорошие и подойдут для классификации.

Есть несколько способоб сделать векторы ещё лучше.


Про них можно почитать вот тут:

https://github.com/nlptown/sentence-similarity/blob/master/Simple%20Sentence%20Similarity.ipynb